In [2]:
!pip install wandb

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import wandb
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].



In [68]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2. Define a Convolutional Neural Network
Copy the neural network from the Neural Networks section before and modify it to
take 3-channel images (instead of 1-channel images as it was defined).



In [86]:
class Net(nn.Module):
    def __init__(self, num_filters):
        super().__init__()
        self.conv1 = nn.Conv2d(3, num_filters, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters, num_filters*2, 5)
        self.fc1 = nn.Linear(num_filters*2 * 5 * 5, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        return x
class NetPool(nn.Module):
    def __init__(self, num_filters=96):
        super().__init__()
        self.conv1 = nn.Conv2d(3, num_filters, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters, num_filters*2, 5)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(num_filters*2, 10)

    def forward(self, x):
        num_filters = self.conv1.out_channels
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.global_pool(x)
        x = x.view(-1, num_filters*2)
        x = nn.functional.relu(self.fc1(x))
        return x


class NetRelu(nn.Module):
    def __init__(self, num_filters=96):
        super().__init__()
        self.conv1 = nn.Conv2d(3, num_filters, 5)
        self.fc1 = nn.Linear(num_filters * 28 * 28, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x



class NetNon(nn.Module):
    def __init__(self, num_filters=110):
        super().__init__()
        self.conv1 = nn.Conv2d(3, num_filters, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters, num_filters*2, 5)
        self.fc1 = nn.Linear(num_filters*2 * 5 * 5, 10)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x



class NetNonBig(nn.Module):
    def __init__(self, num_filters=96):
        super().__init__()
        self.conv1 = nn.Conv2d(3, num_filters, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters, num_filters)
        self.conv3 = nn.Conv2d(num_filters, num_filters)
        self.conv4 = nn.Conv2d(num_filters, num_filters)
        self.fc1 = nn.Linear(num_filters*2 * 5 * 5, 10)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        print(f'NetNonBig shape is {x.shape}')
        x = self.pool(self.conv4(x))
        print(f'NetNonBig shape is {x.shape}')
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x



### 3. Define a Loss function and optimizer
Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [74]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_filters_list = [110]
# 6, 12, 24, 48, 96

### 4. Train the network

This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



In [84]:
def train(trainloader, testloader, num_epochs=5):
  results = {}
  
  for num_filters in num_filters_list:
      # Define the model, optimizer, and loss function
      # model = NetNon(num_filters)

      model = NetPool()
      model = model.to(device)

      optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
      criterion = nn.CrossEntropyLoss()

      wandb.watch(model, criterion, log="all", log_freq=5)

      # Train the model
      num_iter = 0
      for epoch in range(num_epochs):
          wandb.log({"epoch": epoch}, step=num_iter)
          running_loss = 0.0
          for i, data in enumerate(trainloader, 0):
              num_iter += 1
              inputs, labels = data[0].to(device), data[1].to(device)
              optimizer.zero_grad()
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              running_loss += loss.item()
              if i % 200 == 199:    # print every 20 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
                wandb.log({"running loss {}".format(num_filters): running_loss / 200, }, step=num_iter)
                # wandb.log({'Num Filters': num_filters, 'Train Loss': running_loss/20})
                running_loss = 0.0

          epoch_loss = running_loss / len(trainloader)
          wandb.log({f'Num Filters: {num_filters}, epoch Loss': epoch_loss})
  test_model(model, criterion, num_filters)

In [56]:
def test_model(model, criterion, num_filters):

  # Evaluate the model on the test set
  correct = 0
  total = 0
  test_loss = 0.0
  with torch.no_grad():
      for i, data in enumerate(testloader, 0):
          inputs, labels = data[0].to(device), data[1].to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          if i % 20 == 19:    # print every 2000 mini-batches
              print(f'test loss :  {test_loss / 20}' )
              wandb.log({
            'Num Filters': num_filters,
            'Test Loss':test_loss /20})
              test_loss = 0.0

          
  test_accuracy = 100 * correct / total
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f'accuracy is : {test_accuracy}')
  wandb.log({f'Num Filters: {num_filters}, Test Accuracy': test_accuracy, f'Num Filters: {num_filters}, Num Params': num_params})


In [87]:
wandb.init(project='ex_1nn')
train(trainloader, testloader)

[1,   200] loss: 2.285
[1,   400] loss: 2.258
[1,   600] loss: 2.223
[1,   800] loss: 2.229
[1,  1000] loss: 2.171
[1,  1200] loss: 2.153
[1,  1400] loss: 2.128
[1,  1600] loss: 2.109
[1,  1800] loss: 2.132
[1,  2000] loss: 2.097
[1,  2200] loss: 2.106
[1,  2400] loss: 2.044
[1,  2600] loss: 2.078
[1,  2800] loss: 2.043
[1,  3000] loss: 2.017
[1,  3200] loss: 1.966
[1,  3400] loss: 2.015
[1,  3600] loss: 2.001
[1,  3800] loss: 1.930
[1,  4000] loss: 1.910
[1,  4200] loss: 1.891
[1,  4400] loss: 1.849
[1,  4600] loss: 1.893
[1,  4800] loss: 1.855
[1,  5000] loss: 1.850
[1,  5200] loss: 1.787
[1,  5400] loss: 1.790
[1,  5600] loss: 1.761
[1,  5800] loss: 1.768
[1,  6000] loss: 1.711
[1,  6200] loss: 1.668
[1,  6400] loss: 1.731
[1,  6600] loss: 1.697
[1,  6800] loss: 1.703
[1,  7000] loss: 1.663
[1,  7200] loss: 1.665
[1,  7400] loss: 1.691
[1,  7600] loss: 1.583
[1,  7800] loss: 1.646
[1,  8000] loss: 1.611
[1,  8200] loss: 1.545
[1,  8400] loss: 1.649
[1,  8600] loss: 1.547
[1,  8800] 